In [45]:
import os
import ast
import math
import json
import time
import pandas as pd
from dotenv import load_dotenv
import prompt_templates_summarization as pts
from langchain.schema import BaseOutputParser
from langchain.prompts.chat import ChatPromptTemplate
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory
)
from json.decoder import JSONDecodeError
from google.generativeai.types import BlockedPromptException
from google.generativeai.types.generation_types import StopCandidateException

In [46]:
country = "Portugal"
path2SP = "/Users/ctoruno/OneDrive - World Justice Project/EU Subnational"

## Loading API key

In [47]:
load_dotenv()
GoogleAI_key = os.getenv("googleAI_API_key")
os.environ['GOOGLE_API_KEY'] = GoogleAI_key

## Loading data

In [48]:
# Defining a functions that displays the entire pandas data frame:
def showEverything(df):
    with pd.option_context('display.max_rows', None,
                        'display.max_columns', None,
                        'display.width', 1000,
                        'display.precision', 3,
                        'display.colheader_justify', 'left'):
        display(df)

In [59]:
country_data      = pd.read_parquet(f"{path2SP}/EU-S Data/Automated Qualitative Checks/Data/data-classification-1/0_compiled/{country}_classified.parquet.gzip")
location_variable = f"location_{country}"
subset_data       = (
    country_data.copy()
    .loc[(country_data[location_variable] == True) & (country_data["topic_related"].str.contains("Related|Justice|Governance"))]
)
subset_data.shape

(4217, 52)

In [61]:
pillar_columns = subset_data.filter(like='pillar_')
pillar_sum = pillar_columns.sum(axis=0)
pillar_sum

pillar_1    3271
pillar_2    1782
pillar_3     546
pillar_4    2043
pillar_5     353
pillar_6     785
pillar_7    1117
pillar_8    1755
dtype: int64

## Defining chain

In [51]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
}

In [52]:
class JSONOutputParser(BaseOutputParser):
    def parse(self, text: str):
        """
        Parse the output of an LLM call to a valid JSON format.
        """
        return json.loads(text.replace('```json', '').replace('```', ''), strict=False)

In [54]:
def summarize_article(headline, summary, body, pillar):
    """
    This function takes a headline, a summary, and the content of a news article and it sends a call to Google's Gemini
    to summarize a article and provide an impact score focusing on a specific pillar of the Rule of Law.
    """
    
    idx = str(pillar)

    # Setting up the Prompt Template
    chat_prompt = ChatPromptTemplate.from_messages([
                    ("system", pts.context),
                    ("human", pts.instructions)
                ])

    # Defining our chain
    chain_gemini = chat_prompt | ChatGoogleGenerativeAI(model = "gemini-pro",
                                                        temperature = 0.1, 
                                                        safety_settings = safety_settings,
                                                        convert_system_message_to_human = True) | JSONOutputParser()
    
    try: 
        llm_response = chain_gemini.invoke({
            "headline"       : headline,
            "summary"        : summary,
            "body"           : body,
            "pillar_name"    : pts.pillar_names[idx],
            "pillar_bullets" : pts.pillar_bullets[idx]
        })
        status = True
        time.sleep(1)   # We need to slow down the calls. given that the Gemini API has a limit of 60 calls per second

    # The API can still block some of our prompts due to undefined reasons. Sadly, we can't do anything about it, so we
    # predefine the outcome    
    except (BlockedPromptException, StopCandidateException):
        print("Prompt BLOCKED")
        status = False
    
    except JSONDecodeError:
        print("Decode error... trying again...")
        try: 
            llm_response = chain_gemini.invoke({
                "headline"       : headline,
                "summary"        : summary,
                "body"           : body,
                "pillar_name"    : pts.pillar_names[idx],
                "pillar_bullets" : pts.pillar_bullets[idx]
            })
            status = True
            time.sleep(1)

        except JSONDecodeError:
            print("Failed. Skipping article.")
            status = False

    # We use the STATUS variable to throw an outcome to our call depending if our prompt was blocked or not
    if status == True:
        outcome = [llm_response["summary"], llm_response["impact_score"]]

    else:
        outcome = "Skipped article"

    return outcome

## Sending calls in sets and batches

In [62]:
try:
    os.mkdir(f"{path2SP}/EU-S Data/Automated Qualitative Checks/Data/data-summarization/{country}")
    print("Directory created")
except FileExistsError:
    print("Directory already exists")
try:
    for p in range(1,9):
        os.mkdir(f"{path2SP}/EU-S Data/Automated Qualitative Checks/Data/data-summarization/{country}/pillar_{p}")
    print("Sub-directories created")
except FileExistsError:
    print("Sub-directories already exists")

Directory already exists
Sub-directories already exists


In [63]:
for p in range(1,9):

    print("=========================")
    print(f"Starting with PILLAR {p}")
    print("=========================")

    pillar_subset = (
        subset_data.copy()
        .loc[subset_data[f"pillar_{p}"] == True]
    )
    nsets = math.ceil(len(pillar_subset)/1000)

    for set in range(1, nsets+1):
        print("==============================================")
        print(f"Starting with SET {set} out of {nsets} set(s)")
        print("==============================================")

        results = []

        for batch_number in range(1,11):
            starting_row = ((set-1)*1000)+((batch_number-1)*100)
            end_row      = starting_row+100
            batch_subset = pillar_subset.copy().iloc[starting_row:end_row]

            if len(batch_subset) > 0 :
                print("============================================================================")
                print(f"Sending batch number: {batch_number}, start: {starting_row}, end: {end_row}")
                print("============================================================================")

                batch_subset[["summary", "impact_score"]] = batch_subset.apply(lambda row: pd.Series(summarize_article(
                    row["title_trans"], 
                    row["description_trans"], 
                    row["content_trans"], 
                    p
                )), axis = 1)
                results.append(batch_subset)

        # Collapsing and saving data
        collapsed_data = pd.concat(results).drop_duplicates(subset="id")
        collapsed_data.loc[collapsed_data["impact_score"] == "N/A", "impact_score"] = 0
        collapsed_data["impact_score"] = collapsed_data["impact_score"].fillna(0).astype(int)
        collapsed_data.to_parquet(f"{path2SP}/EU-S Data/Automated Qualitative Checks/Data/data-summarization/{country}/pillar_{p}/{country}_pillar_{p}_set_{set}.parquet.gzip", compression="gzip")
        time.sleep(5)

Starting with PILLAR 1
Starting with SET 1 out of 4 set(s)
Sending batch number: 1, start: 0, end: 100


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Sending batch number: 2, start: 100, end: 200
Prompt BLOCKED
Sending batch number: 3, start: 200, end: 300
Sending batch number: 4, start: 300, end: 400
Sending batch number: 5, start: 400, end: 500
Sending batch number: 6, start: 500, end: 600
Sending batch number: 7, start: 600, end: 700
Prompt BLOCKED
Sending batch number: 8, start: 700, end: 800
Prompt BLOCKED
Sending batch number: 9, start: 800, end: 900
Sending batch number: 10, start: 900, end: 1000
Prompt BLOCKED
Starting with SET 2 out of 4 set(s)
Sending batch number: 1, start: 1000, end: 1100
Sending batch number: 2, start: 1100, end: 1200
Sending batch number: 3, start: 1200, end: 1300
Sending batch number: 4, start: 1300, end: 1400
Prompt BLOCKED
Sending batch number: 5, start: 1400, end: 1500
Sending batch number: 6, start: 1500, end: 1600
Prompt BLOCKED
Sending batch number: 7, start: 1600, end: 1700
Prompt BLOCKED
Prompt BLOCKED
Sending batch number: 8, start: 1700, end: 1800
Sending batch number: 9, start: 1800, end: 1

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Prompt BLOCKED
Prompt BLOCKED
Sending batch number: 7, start: 600, end: 700
Prompt BLOCKED
Sending batch number: 8, start: 700, end: 800


...

Sending batch number: 9, start: 800, end: 900
Sending batch number: 10, start: 900, end: 1000
Starting with SET 2 out of 2 set(s)
Sending batch number: 1, start: 1000, end: 1100
Prompt BLOCKED
Prompt BLOCKED
Prompt BLOCKED
Sending batch number: 2, start: 1100, end: 1200
Starting with PILLAR 8
Starting with SET 1 out of 2 set(s)
Sending batch number: 1, start: 0, end: 100
Prompt BLOCKED
Sending batch number: 2, start: 100, end: 200
Prompt BLOCKED
Prompt BLOCKED


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Sending batch number: 3, start: 200, end: 300
Prompt BLOCKED
Sending batch number: 4, start: 300, end: 400
Prompt BLOCKED
Sending batch number: 5, start: 400, end: 500
Sending batch number: 6, start: 500, end: 600
Prompt BLOCKED
Decode error... trying again...
Prompt BLOCKED
Sending batch number: 7, start: 600, end: 700
Sending batch number: 8, start: 700, end: 800
Prompt BLOCKED
Prompt BLOCKED
Prompt BLOCKED
Sending batch number: 9, start: 800, end: 900
Prompt BLOCKED
Prompt BLOCKED
Sending batch number: 10, start: 900, end: 1000
Prompt BLOCKED
Prompt BLOCKED
Prompt BLOCKED
Prompt BLOCKED
Starting with SET 2 out of 2 set(s)
Sending batch number: 1, start: 1000, end: 1100
Sending batch number: 2, start: 1100, end: 1200
Prompt BLOCKED
Sending batch number: 3, start: 1200, end: 1300


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Sending batch number: 4, start: 1300, end: 1400
Sending batch number: 5, start: 1400, end: 1500
Sending batch number: 6, start: 1500, end: 1600
Sending batch number: 7, start: 1600, end: 1700
Prompt BLOCKED
Prompt BLOCKED
Sending batch number: 8, start: 1700, end: 1800
Prompt BLOCKED
Prompt BLOCKED
